# Métodos numéricos para resolver el problema de enfriamiento de Newton

Al sacar un pastel del horno, su temperatura es 160 °C. Tres minutos depués,
su temperatura es de 90°C. ¿Cuánto tiempo le tomará al pastel enfriarse hasta
la temperatura de 30 °C?. Considere una temperatura ambiente de 20 °C.

Se presentan los siguientes métodos:
* Euler
* Euler modificado
* Runge-Kutta de orden 4

Si no tienes instaladas las bibliotecas necesarias, ejecuta la celda 'Instalar bibliotecas'. Si estas trabajando en el entorno se jupyterhub que ya esta configurado, no es necesario realizar este paso.

In [2]:
# Instalar bibliotecas
!pip install metodos_numericos_dcb_fi -U -q

Para importar las bibliotecas necesarias, ejecuta la celda 'Importar las bibliotecas'.

In [3]:
# Importar las bibliotecas
from sympy import lambdify, sympify # ¡¡¡NO MODIFICAR!!!
from sympy.abc import t, T # ¡¡¡NO MODIFICAR!!!
import plotly.express as px # ¡¡¡NO MODIFICAR!!!
import plotly.graph_objects as go # ¡¡¡NO MODIFICAR!!!
import numpy as np # ¡¡¡NO MODIFICAR!!!
import metodos_numericos_dcb_fi.utilidades as ut # ¡¡¡NO MODIFICAR!!!
from ipywidgets import interact, FloatText, IntText # ¡¡¡NO MODIFICAR!!!

### Caracterisación del problema, definicion del modelo matemático y constantes para los métodos numéricos

En esta celda puedes editar los parámetros y condificiones del problema. Es importante que la ejecutes al principio para que los cambios tengan efecto en todo el código. Si quieres volver a los valores originales, solo tienes que ejecutar esta celda nuevamente.

In [ ]:
global t_0, t_f, valores_t, t_ambiente, k, C, f_real, f_sol_analitica, h, n_pasos, y_0
def modificar_valores(t_0_u, t_f_u, t_ambiente_u, k_u, h_u):
    global t_0, t_f, valores_t, t_ambiente, k, C, f_real, f_sol_analitica, h, n_pasos, y_0
    # valores de tiempo
    ut.validarTiempo(t_0_u, t_f_u)
    t_0 = t_0_u # tiempo inicial en segundos
    t_f = t_f_u # tiempo final en segundos
    valores_t = np.linspace(t_0, t_f, (t_f-t_0)*15) # vector de tiempo

    # caracterizacion del problema
    k = k_u # constante de enfriamiento
    t_ambiente = t_ambiente_u # temperatura ambiente

    # valores de las ecuaciones
    C = 140 #! <- Como se obtiene este valor?
    f_real = ut.convertirFuncion(f'-{k}*(T-{t_ambiente})', 'T')
    f_sol_analitica = ut.convertirFuncion(f'{C}*exp(-{k}*t)+{t_ambiente}', 't')

    # valores para los metodos numericos
    h = h_u # tamaño del paso
    n_pasos = int((t_f-t_0)/h) # numero de pasos
    y_0 = f_sol_analitica.f(t_0) # Velocidad inicial

interact(
    modificar_valores,
    t_0_u=IntText(value=0, description='Tiempo inicial (m):', style={'description_width': 'initial'}),
    t_f_u=IntText(value=25, description='Tiempo final (m):', style={'description_width': 'initial'}),
    t_ambiente_u=FloatText(value=20, description='T ambiente'),
    k_u=FloatText(value=0.23105, description='k'),
    h_u=FloatText(value=2, description='Tamaño de paso (min):', style={'description_width': 'initial'})
)

### Solucion analitica

Para ver la solucion analitica del problema, ejecuta la siguiente celda. Es importante ejecutar esta celda para que este disponible en las siguientes celdas.

In [ ]:
def solucion_analitica(f_, valores_t, C, k, mostrar=True):
    f = f_.f # Obtener la funcion evaluable
    valores_y = list(map(lambda t: f(t), valores_t)) # Evaluar la función en el vector de tiempo

    # Graficar
    titulo = f'Enfriamiento de Newton<br>Solucion analítica: T(t)={round(C,3)}exp(-{round(k,3)}t)+{round(t_ambiente,3)}' # Titulo de la gráfica
    eje_x = 't[min]' # Nombre del eje x
    eje_y = 'T(t)[C]' # Nombre del eje y 

    return ut.graficar(valores=[valores_t, valores_y], titulo=titulo, labels=[eje_x, eje_y], metodo='Analitica', mostrar=mostrar) # Regresar gráfica de la solución analítica

plot_Analitica = solucion_analitica(f_sol_analitica, valores_t, C, k) # Graficar solución analítica

### Euler

En esta celda se codifica el método de Euler. Ejecuta la celda para que este  disponible en las siguientes celdas.

In [ ]:
def metodo_euler(f_, t_0, y_0, h, n_pasos, mostrar=True):
    f = f_.f # Obtener la función evaluable
    valores_y_Euler = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo
    
    # Iterar sobre el número de pasos y aplicar el método de Euler
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h) # Agregar el valor de t a la lista
        y_nueva = valores_y_Euler[i] + h * f(valores_y_Euler[i]) # Método de Euler
        valores_y_Euler.append(y_nueva) # Agregar el valor de y a la lista

    # Graficar
    titulo = f'Enfriamiento de Newton<br>Método de Euler con h = {round(h,3)}, n = {n_pasos} y T(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[min]' # Nombre del eje x
    eje_y = 'T(t)[C]' # Nombre del eje y

    return ut.graficar(valores=[valores_t, valores_y_Euler], titulo=titulo, labels=[eje_x, eje_y], metodo='Euler', mostrar=mostrar) # ¡¡¡NO MODIFICAR!!!

grafica_metodo_euler=metodo_euler(f_real, t_0, y_0, h, n_pasos) # Graficar método de Euler

### Euler modificado

En esta celda se codifica el método de Euler modificado. Ejecuta la celda para que este  disponible en las siguientes celdas.

In [ ]:
def metodo_euler_mod(f_, t_0, y_0, h, n_pasos, mostrar=True):
    f = f_.f # Obtener la función evaluable
    valores_y_Euler_Mod = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo

    # Iterar sobre el número de pasos y aplicar el método de Euler modificado
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h)
        y_nueva = valores_y_Euler_Mod[i] + h * f(valores_y_Euler_Mod[i] + h/2 * f(valores_y_Euler_Mod[i])) # ¡¡¡NO MODIFICAR!!!
        valores_y_Euler_Mod.append(y_nueva) # ¡¡¡NO MODIFICAR!!!

    # Graficar
    titulo = f'Enfriamiento de Newton<br>Método de Euler modificado con h = {round(h,3)}, n = {n_pasos} y N(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[min]' # Nombre del eje x
    eje_y = 'T(t)[C]' # Nombre del eje y

    return ut.graficar(valores=[valores_t, valores_y_Euler_Mod], titulo=titulo, labels=[eje_x, eje_y], metodo='Euler M', mostrar=mostrar) # ¡¡¡NO MODIFICAR!!!

grafica_Euler_Mod = metodo_euler_mod(f_real, t_0, y_0, h, n_pasos) # Graficar método de Euler modificado

## Runge-Kutta de orden 4

En esta celda se codifica el método de Rung-Kutta de 4 orden. Ejecuta la celda para que este  disponible en las siguientes celdas.

In [ ]:
def metodo_RK_4(f_, t_0, y_0, h, n_pasos, mostrar=True):
    f = f_.f # Obtener la función evaluable
    valores_y_RK_4 = [y_0] # Lista de valores de y
    valores_t = [t_0] # Vector de tiempo

    # Iterar sobre el número de pasos y aplicar el método de Runge-Kutta 4
    for i in range(n_pasos):
        valores_t.append(valores_t[i] + h) # ¡¡¡NO MODIFICAR!!!
        k1 = h * f(valores_y_RK_4[i]) # ¡¡¡NO MODIFICAR!!!
        k2 = h * f(valores_y_RK_4[i] + k1/2) # ¡¡¡NO MODIFICAR!!!
        k3 = h * f(valores_y_RK_4[i] + k2/2) # ¡¡¡NO MODIFICAR!!!
        k4 = h * f(valores_y_RK_4[i] + k3) # ¡¡¡NO MODIFICAR!!!
        y_nueva = valores_y_RK_4[i] + 1/6 * (k1 + 2*k2 + 2*k3 + k4) # ¡¡¡NO MODIFICAR!!!
        valores_y_RK_4.append(y_nueva) # ¡¡¡NO MODIFICAR!!!

    # Graficar
    titulo = f'Enfriamiento de Newton<br>Método de Runge-Kutta 4 con h = {round(h,3)}, n = {n_pasos} y N(0) = {round(y_0,3)}' # ¡¡¡NO MODIFICAR!!!
    eje_x = 't[min]' # Nombre del eje x
    eje_y = 'T(t)[C]' # Nombre del eje y

    return ut.graficar(valores=[valores_t, valores_y_RK_4], titulo=titulo, labels=[eje_x, eje_y], metodo='RK4', mostrar=mostrar) # ¡¡¡NO MODIFICAR!!!

grafica_RK_4 = metodo_RK_4(f_real, t_0, y_0, h, n_pasos) # Graficar método de Runge-Kutta 4

## Comparacion de los metodos

En esta celda se comparan los resultados obtenidos con los diferentes métodos numéricos. Para que funcione debes haber ejecutado la celda de solucion analitica y las 3 celdas donde se codifican los métodos.

In [ ]:
titulo = f'Enfriamiento de Newton, comparación de métodos'
labels = ['t[min]', 'T(t)[C]']

def compararResultados():
    plot_Analitica= solucion_analitica(f_sol_analitica, valores_t, C, k, mostrar=False)
    plot_Euler = metodo_euler(f_real, t_0, y_0, h, n_pasos, mostrar=False)
    plot_Euler_Mod = metodo_euler_mod(f_real, t_0, y_0, h, n_pasos, mostrar=False)
    plot_RK_4 = metodo_RK_4(f_real, t_0, y_0, h, n_pasos, mostrar=False)
    return ut.compararMetodos(titulo, plot_Analitica, plot_Euler, plot_Euler_Mod, plot_RK_4, labels=labels) # Comparar métodos ¡¡¡NO MODIFICAR!!!

compararResultados()

### Videos de apoyo

Ejecuta la siguiente celda para ver el video recomendado.

In [ ]:
from IPython.display import YouTubeVideo
ytv = YouTubeVideo('M7IO8cA8J5M')
display(ytv)

### Referencias

[1] Chapra, S. C., & Canale, R. P. (2011). Métodos numéricos para ingenieros (6.a ed.) [Electrónico]. [enlace](https://eds.s.ebscohost.com/eds/detail/detail?vid=2&sid=5ad28e1c-ae1c-4a2c-99e4-bd280e8b1618%40redis&bdata=Jmxhbmc9ZXMmc2l0ZT1lZHMtbGl2ZQ%3d%3d#AN=lib.MX001001698818&db=cat02025a)

[2] Carnahan, B., Luther, H. A., & Wilkes, J. O. (1990). Applied Numerical methods. Krieger Publishing Company. 
[Ubicación Facultad de Ingeniería, UNAM](https://eds.s.ebscohost.com/eds/detail/detail?vid=2&sid=7bf807f7-00a8-433e-ab55-6e0dbb10d8fd%40redis&bdata=Jmxhbmc9ZXMmc2l0ZT1lZHMtbGl2ZQ%3d%3d#AN=lib.MX001000592290&db=cat02025a)